In [1]:
from typing import List
from pydantic import BaseModel, Field

from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
class Product(BaseModel):
    """
    Represents a product with various attributes such as ID, name, description, price, discount, and URL.
    """
    product_id: str = Field(..., description="A unique identifier for the product")
    product_name: str = Field(..., description="The name of the product")
    product_description: str = Field(..., description="A summary passage describes product")
    price: float = Field(..., description="The price of the product")
    discount: float = Field(..., description="The discount on the product, ranging from 0 to 80%")
    url: str = Field(..., description="Dummy url directs to product website")
    
    
class Products(BaseModel):
    product: List[Product] = Field(list, min_length=100, max_length=120)
    
GENERATE_DATA_PROMPT = """
You are an expert in data generation, you are tasked with generating product data of a supermarket.

You must strictly follow these instructions to generate data best to user expectation:
1. Generate data to json as following format:
```json
[
    {{
        "product_id": <product_id>,
        "product_name": <product_name>,
        "product_description": <product_description>,
        "price": <price>,
        "discount": <discount, from 0-80%>
        "url": <url direct to product website>
    }},
    ...,
    <product_n>
]
```
2. All generated data should be in Vietnamese,
3. Generate at least 100 items.
4. You must generate data related to this {category} category
5. **Only return generated data without additional comment or text.**
"""

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
# structured_llm = llm.with_structured_output(Products)
prompt = ChatPromptTemplate.from_messages([
    ("system", GENERATE_DATA_PROMPT),
    ("human", "Now let help me generate data as I described")
])

chain = prompt | llm

In [3]:
import asyncio

categories = [
    "RAU - CỦ - QUẢ",
    "THỊT",
    "THUỶ HẢI SẢN",
    "BƠ - SỮA - TRỨNG",
    "BÁNH TƯƠI",
    "BÁNH - KẸO - NGŨ CỐC",
    "THỰC PHẨM CHẾ BIẾN",
    "THỰC PHẨM ĐÔNG LẠNH",
    "NHU YẾU PHẨM & ĐỒ HỘP",
    "RƯỢU & NƯỚC GIẢI KHÁT",
    "THIẾT BỊ ĐIỆN - ĐIỆN TỬ & VĂN PHÒNG PHẨM",
    "VẬT DỤNG GIA ĐÌNH & VẬT PHẨM TRANG TRÍ",
    "HÓA MỸ PHẨM",
    "MAY MẶC & GIÀY DA"
]

res = []
generate_data = await asyncio.gather(*[chain.ainvoke({"category": category}) for category in categories])

In [5]:
import json_repair

results = []
for category, msg in zip(categories, generate_data):
    json_data = json_repair.loads(msg.content)
    results.append(
        {
            "categories": category,
            "products": json_data
        }
    )

In [8]:
import json

with open("../data/products.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

In [10]:
Now let help me generate a dataset with 1000 products contains these categories:

as following format:
```json
{
    "category": <category_name>,
    "products": [
        {
            "product_id": <product_id>,
            "product_name": <product_name>,
            "product_description": <product_description>,
            "price": <price>,
            "discount": <discount, from 0-80%>
            "url": <url direct to product website>
        }
    ]
}
```


**Notes**:
    - all generated data is in vietnamese
    - product descriptions must be a long, detailed passage about product

SyntaxError: invalid syntax (349922083.py, line 1)

In [ ]:
[
    {
        "category": "RAU - CỦ - QUẢ",
        "products": [
            {
                "product_id": "PROD0001",
                "product_name": "Củ dền đỏ",
                "product_description": "Củ dền đỏ của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Củ dền đỏ là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình.",
                "price": 38000,
                "discount": "40%",
                "url": "https://example.com/san-pham/rau-cu-qua/cu-den-do-PROD0001.html"
            },
            {
                "product_id": "PROD0002",
                "product_name": "Nho xanh không hạt",
                "product_description": "Nho xanh không hạt của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Nho xanh không hạt là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Hãy thêm Nho xanh không hạt vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn!",
                "price": 125000,
                "discount": "28%",
                "url": "https://example.com/san-pham/rau-cu-qua/nho-xanh-khong-hat-PROD0002.html"
            },
            {
                "product_id": "PROD0003",
                "product_name": "Khoai lang mật",
                "product_description": "Khoai lang mật của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Khoai lang mật là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào.",
                "price": 56000,
                "discount": "65%",
                "url": "https://example.com/san-pham/rau-cu-qua/khoai-lang-mat-PROD0003.html"
            },
            {
                "product_id": "PROD0004",
                "product_name": "Cà rốt tươi",
                "product_description": "Cà rốt tươi của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Cà rốt tươi là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Hãy thêm Cà rốt tươi vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn!",
                "price": 28000,
                "discount": "60%",
                "url": "https://example.com/san-pham/rau-cu-qua/ca-rot-tuoi-PROD0004.html"
            },
            {
                "product_id": "PROD0005",
                "product_name": "Bắp cải thảo",
                "product_description": "Bắp cải thảo của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Bắp cải thảo là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày. Hãy thêm Bắp cải thảo vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn!",
                "price": 31000,
                "discount": "15%",
                "url": "https://example.com/san-pham/rau-cu-qua/bap-cai-thao-PROD0005.html"
            },
            {
                "product_id": "PROD0006",
                "product_name": "Khoai tây chiên giòn",
                "product_description": "Khoai tây chiên giòn của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Khoai tây chiên giòn là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình.",
                "price": 60000,
                "discount": "40%",
                "url": "https://example.com/san-pham/rau-cu-qua/khoai-tay-chien-gion-PROD0006.html"
            },
            {
                "product_id": "PROD0007",
                "product_name": "Bắp non",
                "product_description": "Bắp non của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Bắp non là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày. Hãy thêm Bắp non vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn!",
                "price": 31000,
                "discount": "65%",
                "url": "https://example.com/san-pham/rau-cu-qua/bap-non-PROD0007.html"
            },
            {
                "product_id": "PROD0008",
                "product_name": "Rau muống",
                "product_description": "Rau muống của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Rau muống là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào.",
                "price": 20000,
                "discount": "21%",
                "url": "https://example.com/san-pham/rau-cu-qua/rau-muong-PROD0008.html"
            },
            {
                "product_id": "PROD0009",
                "product_name": "Xà lách thủy canh",
                "product_description": "Xà lách thủy canh của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Xà lách thủy canh là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào.",
                "price": 26000,
                "discount": "71%",
                "url": "https://example.com/san-pham/rau-cu-qua/xa-lach-thuy-canh-PROD0009.html"
            },
            {
                "product_id": "PROD0010",
                "product_name": "Bí đao non",
                "product_description": "Bí đao non của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Bí đao non là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Hãy thêm Bí đao non vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn! Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày.",
                "price": 19000,
                "discount": "11%",
                "url": "https://example.com/san-pham/rau-cu-qua/bi-dao-non-PROD0010.html"
            },
            {
                "product_id": "PROD0011",
                "product_name": "Đu đủ xanh",
                "product_description": "Đu đủ xanh của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Đu đủ xanh là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Hãy thêm Đu đủ xanh vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn! Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào.",
                "price": 29000,
                "discount": "27%",
                "url": "https://example.com/san-pham/rau-cu-qua/du-du-xanh-PROD0011.html"
            },
            {
                "product_id": "PROD0012",
                "product_name": "Rau ngót",
                "product_description": "Rau ngót của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Rau ngót là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào.",
                "price": 23000,
                "discount": "3%",
                "url": "https://example.com/san-pham/rau-cu-qua/rau-ngot-PROD0012.html"
            },
            {
                "product_id": "PROD0013",
                "product_name": "Mướp hương",
                "product_description": "Mướp hương của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Mướp hương là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày. Hãy thêm Mướp hương vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn!",
                "price": 22000,
                "discount": "29%",
                "url": "https://example.com/san-pham/rau-cu-qua/muop-huong-PROD0013.html"
            },
            {
                "product_id": "PROD0014",
                "product_name": "Táo Fuji nhập khẩu",
                "product_description": "Táo Fuji nhập khẩu của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Táo Fuji nhập khẩu là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày.",
                "price": 95000,
                "discount": "52%",
                "url": "https://example.com/san-pham/rau-cu-qua/tao-fuji-nhap-khau-PROD0014.html"
            },
            {
                "product_id": "PROD0015",
                "product_name": "Dưa chuột baby",
                "product_description": "Dưa chuột baby của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Dưa chuột baby là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Hãy thêm Dưa chuột baby vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn! Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng.",
                "price": 27000,
                "discount": "61%",
                "url": "https://example.com/san-pham/rau-cu-qua/dua-chuot-baby-PROD0015.html"
            },
            {
                "product_id": "PROD0016",
                "product_name": "Dứa Queen",
                "product_description": "Dứa Queen của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Dứa Queen là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày. Hãy thêm Dứa Queen vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn! Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào.",
                "price": 45000,
                "discount": "1%",
                "url": "https://example.com/san-pham/rau-cu-qua/dua-queen-PROD0016.html"
            },
            {
                "product_id": "PROD0017",
                "product_name": "Nấm rơm tươi",
                "product_description": "Nấm rơm tươi của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Nấm rơm tươi là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Hãy thêm Nấm rơm tươi vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn!",
                "price": 48000,
                "discount": "1%",
                "url": "https://example.com/san-pham/rau-cu-qua/nam-rom-tuoi-PROD0017.html"
            },
            {
                "product_id": "PROD0018",
                "product_name": "Cà chua Đà Lạt",
                "product_description": "Cà chua Đà Lạt của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Cà chua Đà Lạt là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP. Hãy thêm Cà chua Đà Lạt vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn!",
                "price": 32000,
                "discount": "55%",
                "url": "https://example.com/san-pham/rau-cu-qua/ca-chua-da-lat-PROD0018.html"
            },
            {
                "product_id": "PROD0019",
                "product_name": "Bắp cải tím",
                "product_description": "Bắp cải tím của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Bắp cải tím là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Hãy thêm Bắp cải tím vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn!",
                "price": 35000,
                "discount": "39%",
                "url": "https://example.com/san-pham/rau-cu-qua/bap-cai-tim-PROD0019.html"
            },
            {
                "product_id": "PROD0020",
                "product_name": "Bưởi da xanh",
                "product_description": "Bưởi da xanh của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Bưởi da xanh là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng.",
                "price": 60000,
                "discount": "11%",
                "url": "https://example.com/san-pham/rau-cu-qua/buoi-da-xanh-PROD0020.html"
            },
            {
                "product_id": "PROD0021",
                "product_name": "Rau cải bắp hữu cơ",
                "product_description": "Rau cải bắp hữu cơ của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Rau cải bắp hữu cơ là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Hãy thêm Rau cải bắp hữu cơ vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn! Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng.",
                "price": 30000,
                "discount": "71%",
                "url": "https://example.com/san-pham/rau-cu-qua/rau-cai-bap-huu-co-PROD0021.html"
            },
            {
                "product_id": "PROD0022",
                "product_name": "Hành tây Đà Lạt",
                "product_description": "Hành tây Đà Lạt của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Hành tây Đà Lạt là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào.",
                "price": 25000,
                "discount": "40%",
                "url": "https://example.com/san-pham/rau-cu-qua/hanh-tay-da-lat-PROD0022.html"
            },
            {
                "product_id": "PROD0023",
                "product_name": "Dưa hấu không hạt",
                "product_description": "Dưa hấu không hạt của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Dưa hấu không hạt là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng.",
                "price": 42000,
                "discount": "56%",
                "url": "https://example.com/san-pham/rau-cu-qua/dua-hau-khong-hat-PROD0023.html"
            },
            {
                "product_id": "PROD0024",
                "product_name": "Chuối tiêu xanh",
                "product_description": "Chuối tiêu xanh của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Chuối tiêu xanh là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Hãy thêm Chuối tiêu xanh vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn! Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày.",
                "price": 24000,
                "discount": "70%",
                "url": "https://example.com/san-pham/rau-cu-qua/chuoi-tieu-xanh-PROD0024.html"
            },
            {
                "product_id": "PROD0025",
                "product_name": "Ớt chuông xanh đỏ",
                "product_description": "Ớt chuông xanh đỏ của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Ớt chuông xanh đỏ là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng.",
                "price": 40000,
                "discount": "7%",
                "url": "https://example.com/san-pham/rau-cu-qua/ot-chuong-xanh-do-PROD0025.html"
            },
            {
                "product_id": "PROD0026",
                "product_name": "Su hào",
                "product_description": "Su hào của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Su hào là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày. Hãy thêm Su hào vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn!",
                "price": 25000,
                "discount": "49%",
                "url": "https://example.com/san-pham/rau-cu-qua/su-hao-PROD0026.html"
            },
            {
                "product_id": "PROD0027",
                "product_name": "Cà tím",
                "product_description": "Cà tím của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Cà tím là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng.",
                "price": 28000,
                "discount": "11%",
                "url": "https://example.com/san-pham/rau-cu-qua/ca-tim-PROD0027.html"
            },
            {
                "product_id": "PROD0028",
                "product_name": "Cam sành Bến Tre",
                "product_description": "Cam sành Bến Tre của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Cam sành Bến Tre là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP. Hãy thêm Cam sành Bến Tre vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn! Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày.",
                "price": 45000,
                "discount": "65%",
                "url": "https://example.com/san-pham/rau-cu-qua/cam-sanh-ben-tre-PROD0028.html"
            },
            {
                "product_id": "PROD0029",
                "product_name": "Chanh tươi",
                "product_description": "Chanh tươi của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Chanh tươi là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng.",
                "price": 18000,
                "discount": "27%",
                "url": "https://example.com/san-pham/rau-cu-qua/chanh-tuoi-PROD0029.html"
            },
            {
                "product_id": "PROD0030",
                "product_name": "Củ dền đỏ",
                "product_description": "Củ dền đỏ của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Củ dền đỏ là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Hãy thêm Củ dền đỏ vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn! Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày.",
                "price": 38000,
                "discount": "40%",
                "url": "https://example.com/san-pham/rau-cu-qua/cu-den-do-PROD0030.html"
            },
            {
                "product_id": "PROD0031",
                "product_name": "Nho xanh không hạt",
                "product_description": "Nho xanh không hạt của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Nho xanh không hạt là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng.",
                "price": 125000,
                "discount": "12%",
                "url": "https://example.com/san-pham/rau-cu-qua/nho-xanh-khong-hat-PROD0031.html"
            },
            {
                "product_id": "PROD0032",
                "product_name": "Khoai lang mật",
                "product_description": "Khoai lang mật của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Khoai lang mật là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Hãy thêm Khoai lang mật vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn! Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày.",
                "price": 58000,
                "discount": "4%",
                "url": "https://example.com/san-pham/rau-cu-qua/khoai-lang-mat-PROD0032.html"
            },
            {
                "product_id": "PROD0033",
                "product_name": "Cà rốt tươi",
                "product_description": "Cà rốt tươi của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Cà rốt tươi là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào.",
                "price": 29000,
                "discount": "41%",
                "url": "https://example.com/san-pham/rau-cu-qua/ca-rot-tuoi-PROD0033.html"
            },
            {
                "product_id": "PROD0034",
                "product_name": "Bắp cải thảo",
                "product_description": "Bắp cải thảo của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Bắp cải thảo là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình.",
                "price": 31000,
                "discount": "11%",
                "url": "https://example.com/san-pham/rau-cu-qua/bap-cai-thao-PROD0034.html"
            },
            {
                "product_id": "PROD0035",
                "product_name": "Khoai tây chiên giòn",
                "product_description": "Khoai tây chiên giòn của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Khoai tây chiên giòn là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Hãy thêm Khoai tây chiên giòn vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn! Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng.",
                "price": 60000,
                "discount": "65%",
                "url": "https://example.com/san-pham/rau-cu-qua/khoai-tay-chien-gion-PROD0035.html"
            },
            {
                "product_id": "PROD0036",
                "product_name": "Bắp non",
                "product_description": "Bắp non của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Bắp non là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày.",
                "price": 30000,
                "discount": "56%",
                "url": "https://example.com/san-pham/rau-cu-qua/bap-non-PROD0036.html"
            },
            {
                "product_id": "PROD0037",
                "product_name": "Rau muống",
                "product_description": "Rau muống của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Rau muống là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Hãy thêm Rau muống vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn!",
                "price": 22000,
                "discount": "10%",
                "url": "https://example.com/san-pham/rau-cu-qua/rau-muong-PROD0037.html"
            },
            {
                "product_id": "PROD0038",
                "product_name": "Xà lách thủy canh",
                "product_description": "Xà lách thủy canh của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Xà lách thủy canh là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng.",
                "price": 27000,
                "discount": "75%",
                "url": "https://example.com/san-pham/rau-cu-qua/xa-lach-thuy-canh-PROD0038.html"
            },
            {
                "product_id": "PROD0039",
                "product_name": "Bí đao non",
                "product_description": "Bí đao non của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Bí đao non là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Hãy thêm Bí đao non vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn!",
                "price": 20000,
                "discount": "1%",
                "url": "https://example.com/san-pham/rau-cu-qua/bi-dao-non-PROD0039.html"
            },
            {
                "product_id": "PROD0040",
                "product_name": "Đu đủ xanh",
                "product_description": "Đu đủ xanh của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Đu đủ xanh là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng.",
                "price": 29000,
                "discount": "62%",
                "url": "https://example.com/san-pham/rau-cu-qua/du-du-xanh-PROD0040.html"
            },
            {
                "product_id": "PROD0041",
                "product_name": "Rau ngót",
                "product_description": "Rau ngót của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Rau ngót là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Hãy thêm Rau ngót vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn! Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình.",
                "price": 22000,
                "discount": "40%",
                "url": "https://example.com/san-pham/rau-cu-qua/rau-ngot-PROD0041.html"
            },
            {
                "product_id": "PROD0042",
                "product_name": "Mướp hương",
                "product_description": "Mướp hương của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Mướp hương là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP.",
                "price": 22000,
                "discount": "28%",
                "url": "https://example.com/san-pham/rau-cu-qua/muop-huong-PROD0042.html"
            },
            {
                "product_id": "PROD0043",
                "product_name": "Táo Fuji nhập khẩu",
                "product_description": "Táo Fuji nhập khẩu của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Táo Fuji nhập khẩu là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình.",
                "price": 99000,
                "discount": "3%",
                "url": "https://example.com/san-pham/rau-cu-qua/tao-fuji-nhap-khau-PROD0043.html"
            },
            {
                "product_id": "PROD0044",
                "product_name": "Dưa chuột baby",
                "product_description": "Dưa chuột baby của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Dưa chuột baby là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP. Hãy thêm Dưa chuột baby vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn!",
                "price": 27000,
                "discount": "22%",
                "url": "https://example.com/san-pham/rau-cu-qua/dua-chuot-baby-PROD0044.html"
            },
            {
                "product_id": "PROD0045",
                "product_name": "Dứa Queen",
                "product_description": "Dứa Queen của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Dứa Queen là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình.",
                "price": 45000,
                "discount": "64%",
                "url": "https://example.com/san-pham/rau-cu-qua/dua-queen-PROD0045.html"
            },
            {
                "product_id": "PROD0046",
                "product_name": "Nấm rơm tươi",
                "product_description": "Nấm rơm tươi của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Nấm rơm tươi là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP. Hãy thêm Nấm rơm tươi vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn!",
                "price": 48000,
                "discount": "38%",
                "url": "https://example.com/san-pham/rau-cu-qua/nam-rom-tuoi-PROD0046.html"
            },
            {
                "product_id": "PROD0047",
                "product_name": "Cà chua Đà Lạt",
                "product_description": "Cà chua Đà Lạt của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Cà chua Đà Lạt là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày.",
                "price": 32000,
                "discount": "41%",
                "url": "https://example.com/san-pham/rau-cu-qua/ca-chua-da-lat-PROD0047.html"
            },
            {
                "product_id": "PROD0048",
                "product_name": "Bắp cải tím",
                "product_description": "Bắp cải tím của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Bắp cải tím là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Hãy thêm Bắp cải tím vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn!",
                "price": 35000,
                "discount": "41%",
                "url": "https://example.com/san-pham/rau-cu-qua/bap-cai-tim-PROD0048.html"
            },
            {
                "product_id": "PROD0049",
                "product_name": "Bưởi da xanh",
                "product_description": "Bưởi da xanh của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Bưởi da xanh là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng.",
                "price": 60000,
                "discount": "11%",
                "url": "https://example.com/san-pham/rau-cu-qua/buoi-da-xanh-PROD0049.html"
            },
            {
                "product_id": "PROD0050",
                "product_name": "Rau cải bắp hữu cơ",
                "product_description": "Rau cải bắp hữu cơ của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Rau cải bắp hữu cơ là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào.",
                "price": 31000,
                "discount": "79%",
                "url": "https://example.com/san-pham/rau-cu-qua/rau-cai-bap-huu-co-PROD0050.html"
            },
            {
                "product_id": "PROD0051",
                "product_name": "Hành tây Đà Lạt",
                "product_description": "Hành tây Đà Lạt của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Hành tây Đà Lạt là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày.",
                "price": 26000,
                "discount": "56%",
                "url": "https://example.com/san-pham/rau-cu-qua/hanh-tay-da-lat-PROD0051.html"
            },
            {
                "product_id": "PROD0052",
                "product_name": "Dưa hấu không hạt",
                "product_description": "Dưa hấu không hạt của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Dưa hấu không hạt là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Hãy thêm Dưa hấu không hạt vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn!",
                "price": 44000,
                "discount": "65%",
                "url": "https://example.com/san-pham/rau-cu-qua/dua-hau-khong-hat-PROD0052.html"
            },
            {
                "product_id": "PROD0053",
                "product_name": "Chuối tiêu xanh",
                "product_description": "Chuối tiêu xanh của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Chuối tiêu xanh là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Hãy thêm Chuối tiêu xanh vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn!",
                "price": 25000,
                "discount": "14%",
                "url": "https://example.com/san-pham/rau-cu-qua/chuoi-tieu-xanh-PROD0053.html"
            },
            {
                "product_id": "PROD0054",
                "product_name": "Ớt chuông xanh đỏ",
                "product_description": "Ớt chuông xanh đỏ của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Ớt chuông xanh đỏ là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình.",
                "price": 38000,
                "discount": "4%",
                "url": "https://example.com/san-pham/rau-cu-qua/ot-chuong-xanh-do-PROD0054.html"
            },
            {
                "product_id": "PROD0055",
                "product_name": "Su hào",
                "product_description": "Su hào của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Su hào là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình.",
                "price": 25000,
                "discount": "66%",
                "url": "https://example.com/san-pham/rau-cu-qua/su-hao-PROD0055.html"
            },
            {
                "product_id": "PROD0056",
                "product_name": "Cà tím",
                "product_description": "Cà tím của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Cà tím là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Hãy thêm Cà tím vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn! Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình.",
                "price": 28000,
                "discount": "40%",
                "url": "https://example.com/san-pham/rau-cu-qua/ca-tim-PROD0056.html"
            },
            {
                "product_id": "PROD0057",
                "product_name": "Cam sành Bến Tre",
                "product_description": "Cam sành Bến Tre của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Cam sành Bến Tre là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình.",
                "price": 45000,
                "discount": "38%",
                "url": "https://example.com/san-pham/rau-cu-qua/cam-sanh-ben-tre-PROD0057.html"
            },
            {
                "product_id": "PROD0058",
                "product_name": "Chanh tươi",
                "product_description": "Chanh tươi của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Chanh tươi là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Hãy thêm Chanh tươi vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn! Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào.",
                "price": 18000,
                "discount": "43%",
                "url": "https://example.com/san-pham/rau-cu-qua/chanh-tuoi-PROD0058.html"
            },
            {
                "product_id": "PROD0059",
                "product_name": "Củ dền đỏ",
                "product_description": "Củ dền đỏ của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Củ dền đỏ là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP.",
                "price": 38000,
                "discount": "65%",
                "url": "https://example.com/san-pham/rau-cu-qua/cu-den-do-PROD0059.html"
            },
            {
                "product_id": "PROD0060",
                "product_name": "Nho xanh không hạt",
                "product_description": "Nho xanh không hạt của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Nho xanh không hạt là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP. Hãy thêm Nho xanh không hạt vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn!",
                "price": 120000,
                "discount": "20%",
                "url": "https://example.com/san-pham/rau-cu-qua/nho-xanh-khong-hat-PROD0060.html"
            },
            {
                "product_id": "PROD0061",
                "product_name": "Khoai lang mật",
                "product_description": "Khoai lang mật của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Khoai lang mật là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng.",
                "price": 57000,
                "discount": "38%",
                "url": "https://example.com/san-pham/rau-cu-qua/khoai-lang-mat-PROD0061.html"
            },
            {
                "product_id": "PROD0062",
                "product_name": "Cà rốt tươi",
                "product_description": "Cà rốt tươi của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Cà rốt tươi là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Hãy thêm Cà rốt tươi vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn!",
                "price": 28000,
                "discount": "27%",
                "url": "https://example.com/san-pham/rau-cu-qua/ca-rot-tuoi-PROD0062.html"
            },
            {
                "product_id": "PROD0063",
                "product_name": "Bắp cải thảo",
                "product_description": "Bắp cải thảo của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Bắp cải thảo là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào.",
                "price": 31000,
                "discount": "29%",
                "url": "https://example.com/san-pham/rau-cu-qua/bap-cai-thao-PROD0063.html"
            },
            {
                "product_id": "PROD0064",
                "product_name": "Khoai tây chiên giòn",
                "product_description": "Khoai tây chiên giòn của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Khoai tây chiên giòn là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào.",
                "price": 60000,
                "discount": "73%",
                "url": "https://example.com/san-pham/rau-cu-qua/khoai-tay-chien-gion-PROD0064.html"
            },
            {
                "product_id": "PROD0065",
                "product_name": "Bắp non",
                "product_description": "Bắp non của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Bắp non là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP. Hãy thêm Bắp non vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn!",
                "price": 31000,
                "discount": "3%",
                "url": "https://example.com/san-pham/rau-cu-qua/bap-non-PROD0065.html"
            },
            {
                "product_id": "PROD0066",
                "product_name": "Rau muống",
                "product_description": "Rau muống của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Rau muống là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình.",
                "price": 21000,
                "discount": "10%",
                "url": "https://example.com/san-pham/rau-cu-qua/rau-muong-PROD0066.html"
            },
            {
                "product_id": "PROD0067",
                "product_name": "Xà lách thủy canh",
                "product_description": "Xà lách thủy canh của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Xà lách thủy canh là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Hãy thêm Xà lách thủy canh vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn! Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày.",
                "price": 26000,
                "discount": "10%",
                "url": "https://example.com/san-pham/rau-cu-qua/xa-lach-thuy-canh-PROD0067.html"
            },
            {
                "product_id": "PROD0068",
                "product_name": "Bí đao non",
                "product_description": "Bí đao non của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Bí đao non là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Đảm bảo nguồn gốc rõ ràng, đạt chuẩn VietGAP. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình.",
                "price": 19000,
                "discount": "40%",
                "url": "https://example.com/san-pham/rau-cu-qua/bi-dao-non-PROD0068.html"
            },
            {
                "product_id": "PROD0069",
                "product_name": "Đu đủ xanh",
                "product_description": "Đu đủ xanh của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Đu đủ xanh là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày.",
                "price": 29000,
                "discount": "7%",
                "url": "https://example.com/san-pham/rau-cu-qua/du-du-xanh-PROD0069.html"
            },
            {
                "product_id": "PROD0070",
                "product_name": "Rau ngót",
                "product_description": "Rau ngót của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Rau ngót là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào. Thích hợp để bảo quản trong ngăn mát tủ lạnh từ 5-7 ngày. Hãy thêm Rau ngót vào giỏ hàng để mang đến sự tươi mới cho căn bếp của bạn!",
                "price": 22000,
                "discount": "65%",
                "url": "https://example.com/san-pham/rau-cu-qua/rau-ngot-PROD0070.html"
            },
            {
                "product_id": "PROD0071",
                "product_name": "Mướp hương",
                "product_description": "Mướp hương của chúng tôi được thu hoạch tươi mới mỗi ngày từ các nông trại uy tín, đảm bảo giữ trọn hương vị tự nhiên và giá trị dinh dưỡng cao nhất. Sản phẩm được kiểm định chất lượng nghiêm ngặt, không chứa hóa chất độc hại, an toàn cho sức khỏe cả gia đình. Với màu sắc bắt mắt và độ tươi ngon vượt trội, Mướp hương là lựa chọn lý tưởng cho các bữa ăn hàng ngày, từ salad tươi mát đến món hầm bổ dưỡng. Giàu vitamin C và chất xơ, rất tốt cho hệ tiêu hóa và tăng cường sức đề kháng. Luôn tươi ngon và chất lượng, phù hợp cho mọi gia đình. Có thể chế biến thành nhiều món ngon như salad, nước ép, món luộc hoặc xào.",
                "price": 21000,
                "discount": "75%",
                "url": "https://example.com/san-pham/rau-cu-qua/muop-huong-PROD0071.html"
            }
        ]
    },
    {
        "category": "THỊT",
        "products": [
            {
                "product_id": "PROD0072",
                "product_name": "Thịt đà điểu",
                "product_description": "Thịt đà điểu được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt đà điểu giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein.",
                "price": 357000,
                "discount": "10%",
                "url": "https://example.com/san-pham/thit/thit-da-dieu-PROD0072.html"
            },
            {
                "product_id": "PROD0073",
                "product_name": "Thịt vai bò",
                "product_description": "Thịt vai bò được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt vai bò giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà.",
                "price": 286000,
                "discount": "49%",
                "url": "https://example.com/san-pham/thit/thit-vai-bo-PROD0073.html"
            },
            {
                "product_id": "PROD0074",
                "product_name": "Thịt cốt lết",
                "product_description": "Thịt cốt lết được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt cốt lết giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn.",
                "price": 149000,
                "discount": "44%",
                "url": "https://example.com/san-pham/thit/thit-cot-let-PROD0074.html"
            },
            {
                "product_id": "PROD0075",
                "product_name": "Sườn non heo",
                "product_description": "Sườn non heo được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Sườn non heo giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà.",
                "price": 196000,
                "discount": "3%",
                "url": "https://example.com/san-pham/thit/suon-non-heo-PROD0075.html"
            },
            {
                "product_id": "PROD0076",
                "product_name": "Lưỡi heo",
                "product_description": "Lưỡi heo được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Lưỡi heo giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm.",
                "price": 125000,
                "discount": "51%",
                "url": "https://example.com/san-pham/thit/luoi-heo-PROD0076.html"
            },
            {
                "product_id": "PROD0077",
                "product_name": "Thịt đùi gà góc tư",
                "product_description": "Thịt đùi gà góc tư được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt đùi gà góc tư giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng.",
                "price": 93000,
                "discount": "62%",
                "url": "https://example.com/san-pham/thit/thit-dui-ga-goc-tu-PROD0077.html"
            },
            {
                "product_id": "PROD0078",
                "product_name": "Thịt thăn lợn",
                "product_description": "Thịt thăn lợn được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt thăn lợn giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm.",
                "price": 154000,
                "discount": "11%",
                "url": "https://example.com/san-pham/thit/thit-than-lon-PROD0078.html"
            },
            {
                "product_id": "PROD0079",
                "product_name": "Lòng non heo",
                "product_description": "Lòng non heo được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Lòng non heo giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến.",
                "price": 113000,
                "discount": "65%",
                "url": "https://example.com/san-pham/thit/long-non-heo-PROD0079.html"
            },
            {
                "product_id": "PROD0080",
                "product_name": "Thịt bê non",
                "product_description": "Thịt bê non được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt bê non giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein.",
                "price": 289000,
                "discount": "20%",
                "url": "https://example.com/san-pham/thit/thit-be-non-PROD0080.html"
            },
            {
                "product_id": "PROD0081",
                "product_name": "Tim heo tươi",
                "product_description": "Tim heo tươi được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Tim heo tươi giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein.",
                "price": 105000,
                "discount": "40%",
                "url": "https://example.com/san-pham/thit/tim-heo-tuoi-PROD0081.html"
            },
            {
                "product_id": "PROD0082",
                "product_name": "Ba chỉ bò Mỹ",
                "product_description": "Ba chỉ bò Mỹ được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Ba chỉ bò Mỹ giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn.",
                "price": 320000,
                "discount": "26%",
                "url": "https://example.com/san-pham/thit/ba-chi-bo-my-PROD0082.html"
            },
            {
                "product_id": "PROD0083",
                "product_name": "Gà ta nguyên con",
                "product_description": "Gà ta nguyên con được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Gà ta nguyên con giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến.",
                "price": 178000,
                "discount": "11%",
                "url": "https://example.com/san-pham/thit/ga-ta-nguyen-con-PROD0083.html"
            },
            {
                "product_id": "PROD0084",
                "product_name": "Ức gà tươi CP",
                "product_description": "Ức gà tươi CP được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Ức gà tươi CP giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein.",
                "price": 85000,
                "discount": "65%",
                "url": "https://example.com/san-pham/thit/uc-ga-tuoi-cp-PROD0084.html"
            },
            {
                "product_id": "PROD0085",
                "product_name": "Thịt heo ba chỉ sạch",
                "product_description": "Thịt heo ba chỉ sạch được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt heo ba chỉ sạch giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn.",
                "price": 139000,
                "discount": "22%",
                "url": "https://example.com/san-pham/thit/thit-heo-ba-chi-sach-PROD0085.html"
            },
            {
                "product_id": "PROD0086",
                "product_name": "Thịt dê tơ",
                "product_description": "Thịt dê tơ được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt dê tơ giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn.",
                "price": 310000,
                "discount": "71%",
                "url": "https://example.com/san-pham/thit/thit-de-to-PROD0086.html"
            },
            {
                "product_id": "PROD0087",
                "product_name": "Thịt bò thăn Úc nhập khẩu",
                "product_description": "Thịt bò thăn Úc nhập khẩu được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt bò thăn Úc nhập khẩu giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein.",
                "price": 420000,
                "discount": "11%",
                "url": "https://example.com/san-pham/thit/thit-bo-than-uc-nhap-khau-PROD0087.html"
            },
            {
                "product_id": "PROD0088",
                "product_name": "Thịt vịt xiêm",
                "product_description": "Thịt vịt xiêm được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt vịt xiêm giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng.",
                "price": 145000,
                "discount": "60%",
                "url": "https://example.com/san-pham/thit/thit-vit-xiem-PROD0088.html"
            },
            {
                "product_id": "PROD0089",
                "product_name": "Chân giò heo rút xương",
                "product_description": "Chân giò heo rút xương được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Chân giò heo rút xương giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến.",
                "price": 169000,
                "discount": "55%",
                "url": "https://example.com/san-pham/thit/chan-gio-heo-rut-xuong-PROD0089.html"
            },
            {
                "product_id": "PROD0090",
                "product_name": "Thịt băm hỗn hợp",
                "product_description": "Thịt băm hỗn hợp được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt băm hỗn hợp giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm.",
                "price": 115000,
                "discount": "49%",
                "url": "https://example.com/san-pham/thit/thit-bam-hon-hop-PROD0090.html"
            },
            {
                "product_id": "PROD0091",
                "product_name": "Sườn cừu tươi",
                "product_description": "Sườn cừu tươi được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Sườn cừu tươi giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến.",
                "price": 380000,
                "discount": "71%",
                "url": "https://example.com/san-pham/thit/suon-cuu-tuoi-PROD0091.html"
            },
            {
                "product_id": "PROD0092",
                "product_name": "Bò bắp hoa",
                "product_description": "Bò bắp hoa được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Bò bắp hoa giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn.",
                "price": 270000,
                "discount": "74%",
                "url": "https://example.com/san-pham/thit/bo-bap-hoa-PROD0092.html"
            },
            {
                "product_id": "PROD0093",
                "product_name": "Gan gà",
                "product_description": "Gan gà được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Gan gà giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn.",
                "price": 89000,
                "discount": "52%",
                "url": "https://example.com/san-pham/thit/gan-ga-PROD0093.html"
            },
            {
                "product_id": "PROD0094",
                "product_name": "Thịt bò Kobe",
                "product_description": "Thịt bò Kobe được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt bò Kobe giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein.",
                "price": 490000,
                "discount": "40%",
                "url": "https://example.com/san-pham/thit/thit-bo-kobe-PROD0094.html"
            },
            {
                "product_id": "PROD0095",
                "product_name": "Nạc vai heo",
                "product_description": "Nạc vai heo được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Nạc vai heo giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein.",
                "price": 138000,
                "discount": "20%",
                "url": "https://example.com/san-pham/thit/nac-vai-heo-PROD0095.html"
            },
            {
                "product_id": "PROD0096",
                "product_name": "Dạ dày heo",
                "product_description": "Dạ dày heo được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Dạ dày heo giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng.",
                "price": 110000,
                "discount": "10%",
                "url": "https://example.com/san-pham/thit/da-day-heo-PROD0096.html"
            },
            {
                "product_id": "PROD0097",
                "product_name": "Đùi gà tỏi",
                "product_description": "Đùi gà tỏi được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Đùi gà tỏi giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn.",
                "price": 98000,
                "discount": "1%",
                "url": "https://example.com/san-pham/thit/dui-ga-toi-PROD0097.html"
            },
            {
                "product_id": "PROD0098",
                "product_name": "Thịt đà điểu",
                "product_description": "Thịt đà điểu được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt đà điểu giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến.",
                "price": 360000,
                "discount": "73%",
                "url": "https://example.com/san-pham/thit/thit-da-dieu-PROD0098.html"
            },
            {
                "product_id": "PROD0099",
                "product_name": "Thịt vai bò",
                "product_description": "Thịt vai bò được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt vai bò giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến.",
                "price": 280000,
                "discount": "17%",
                "url": "https://example.com/san-pham/thit/thit-vai-bo-PROD0099.html"
            },
            {
                "product_id": "PROD0100",
                "product_name": "Thịt cốt lết",
                "product_description": "Thịt cốt lết được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt cốt lết giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn.",
                "price": 149000,
                "discount": "65%",
                "url": "https://example.com/san-pham/thit/thit-cot-let-PROD0100.html"
            },
            {
                "product_id": "PROD0101",
                "product_name": "Sườn non heo",
                "product_description": "Sườn non heo được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Sườn non heo giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến.",
                "price": 196000,
                "discount": "40%",
                "url": "https://example.com/san-pham/thit/suon-non-heo-PROD0101.html"
            },
            {
                "product_id": "PROD0102",
                "product_name": "Lưỡi heo",
                "product_description": "Lưỡi heo được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Lưỡi heo giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn.",
                "price": 120000,
                "discount": "3%",
                "url": "https://example.com/san-pham/thit/luoi-heo-PROD0102.html"
            },
            {
                "product_id": "PROD0103",
                "product_name": "Thịt đùi gà góc tư",
                "product_description": "Thịt đùi gà góc tư được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt đùi gà góc tư giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà.",
                "price": 93000,
                "discount": "65%",
                "url": "https://example.com/san-pham/thit/thit-dui-ga-goc-tu-PROD0103.html"
            },
            {
                "product_id": "PROD0104",
                "product_name": "Thịt thăn lợn",
                "product_description": "Thịt thăn lợn được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt thăn lợn giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn.",
                "price": 154000,
                "discount": "65%",
                "url": "https://example.com/san-pham/thit/thit-than-lon-PROD0104.html"
            },
            {
                "product_id": "PROD0105",
                "product_name": "Lòng non heo",
                "product_description": "Lòng non heo được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Lòng non heo giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng.",
                "price": 114000,
                "discount": "56%",
                "url": "https://example.com/san-pham/thit/long-non-heo-PROD0105.html"
            },
            {
                "product_id": "PROD0106",
                "product_name": "Thịt bê non",
                "product_description": "Thịt bê non được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt bê non giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein.",
                "price": 290000,
                "discount": "40%",
                "url": "https://example.com/san-pham/thit/thit-be-non-PROD0106.html"
            },
            {
                "product_id": "PROD0107",
                "product_name": "Tim heo tươi",
                "product_description": "Tim heo tươi được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Tim heo tươi giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến.",
                "price": 108000,
                "discount": "71%",
                "url": "https://example.com/san-pham/thit/tim-heo-tuoi-PROD0107.html"
            },
            {
                "product_id": "PROD0108",
                "product_name": "Ba chỉ bò Mỹ",
                "product_description": "Ba chỉ bò Mỹ được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Ba chỉ bò Mỹ giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein.",
                "price": 320000,
                "discount": "1%",
                "url": "https://example.com/san-pham/thit/ba-chi-bo-my-PROD0108.html"
            },
            {
                "product_id": "PROD0109",
                "product_name": "Gà ta nguyên con",
                "product_description": "Gà ta nguyên con được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Gà ta nguyên con giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm.",
                "price": 178000,
                "discount": "65%",
                "url": "https://example.com/san-pham/thit/ga-ta-nguyen-con-PROD0109.html"
            },
            {
                "product_id": "PROD0110",
                "product_name": "Ức gà tươi CP",
                "product_description": "Ức gà tươi CP được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Ức gà tươi CP giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến.",
                "price": 86000,
                "discount": "49%",
                "url": "https://example.com/san-pham/thit/uc-ga-tuoi-cp-PROD0110.html"
            },
            {
                "product_id": "PROD0111",
                "product_name": "Thịt heo ba chỉ sạch",
                "product_description": "Thịt heo ba chỉ sạch được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt heo ba chỉ sạch giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà.",
                "price": 140000,
                "discount": "71%",
                "url": "https://example.com/san-pham/thit/thit-heo-ba-chi-sach-PROD0111.html"
            },
            {
                "product_id": "PROD0112",
                "product_name": "Thịt dê tơ",
                "product_description": "Thịt dê tơ được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt dê tơ giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng.",
                "price": 310000,
                "discount": "11%",
                "url": "https://example.com/san-pham/thit/thit-de-to-PROD0112.html"
            },
            {
                "product_id": "PROD0113",
                "product_name": "Thịt bò thăn Úc nhập khẩu",
                "product_description": "Thịt bò thăn Úc nhập khẩu được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt bò thăn Úc nhập khẩu giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà.",
                "price": 420000,
                "discount": "14%",
                "url": "https://example.com/san-pham/thit/thit-bo-than-uc-nhap-khau-PROD0113.html"
            },
            {
                "product_id": "PROD0114",
                "product_name": "Thịt vịt xiêm",
                "product_description": "Thịt vịt xiêm được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt vịt xiêm giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến.",
                "price": 140000,
                "discount": "40%",
                "url": "https://example.com/san-pham/thit/thit-vit-xiem-PROD0114.html"
            },
            {
                "product_id": "PROD0115",
                "product_name": "Chân giò heo rút xương",
                "product_description": "Chân giò heo rút xương được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Chân giò heo rút xương giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà.",
                "price": 169000,
                "discount": "40%",
                "url": "https://example.com/san-pham/thit/chan-gio-heo-rut-xuong-PROD0115.html"
            },
            {
                "product_id": "PROD0116",
                "product_name": "Thịt băm hỗn hợp",
                "product_description": "Thịt băm hỗn hợp được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt băm hỗn hợp giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến.",
                "price": 115000,
                "discount": "64%",
                "url": "https://example.com/san-pham/thit/thit-bam-hon-hop-PROD0116.html"
            },
            {
                "product_id": "PROD0117",
                "product_name": "Sườn cừu tươi",
                "product_description": "Sườn cừu tươi được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Sườn cừu tươi giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm.",
                "price": 390000,
                "discount": "71%",
                "url": "https://example.com/san-pham/thit/suon-cuu-tuoi-PROD0117.html"
            },
            {
                "product_id": "PROD0118",
                "product_name": "Bò bắp hoa",
                "product_description": "Bò bắp hoa được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Bò bắp hoa giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến.",
                "price": 270000,
                "discount": "75%",
                "url": "https://example.com/san-pham/thit/bo-bap-hoa-PROD0118.html"
            },
            {
                "product_id": "PROD0119",
                "product_name": "Gan gà",
                "product_description": "Gan gà được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Gan gà giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng.",
                "price": 88000,
                "discount": "14%",
                "url": "https://example.com/san-pham/thit/gan-ga-PROD0119.html"
            },
            {
                "product_id": "PROD0120",
                "product_name": "Thịt bò Kobe",
                "product_description": "Thịt bò Kobe được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt bò Kobe giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà.",
                "price": 490000,
                "discount": "12%",
                "url": "https://example.com/san-pham/thit/thit-bo-kobe-PROD0120.html"
            },
            {
                "product_id": "PROD0121",
                "product_name": "Nạc vai heo",
                "product_description": "Nạc vai heo được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Nạc vai heo giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến.",
                "price": 139000,
                "discount": "65%",
                "url": "https://example.com/san-pham/thit/nac-vai-heo-PROD0121.html"
            },
            {
                "product_id": "PROD0122",
                "product_name": "Dạ dày heo",
                "product_description": "Dạ dày heo được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Dạ dày heo giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng.",
                "price": 110000,
                "discount": "64%",
                "url": "https://example.com/san-pham/thit/da-day-heo-PROD0122.html"
            },
            {
                "product_id": "PROD0123",
                "product_name": "Đùi gà tỏi",
                "product_description": "Đùi gà tỏi được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Đùi gà tỏi giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà.",
                "price": 99000,
                "discount": "15%",
                "url": "https://example.com/san-pham/thit/dui-ga-toi-PROD0123.html"
            },
            {
                "product_id": "PROD0124",
                "product_name": "Thịt đà điểu",
                "product_description": "Thịt đà điểu được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt đà điểu giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein.",
                "price": 360000,
                "discount": "11%",
                "url": "https://example.com/san-pham/thit/thit-da-dieu-PROD0124.html"
            },
            {
                "product_id": "PROD0125",
                "product_name": "Thịt vai bò",
                "product_description": "Thịt vai bò được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt vai bò giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm.",
                "price": 286000,
                "discount": "3%",
                "url": "https://example.com/san-pham/thit/thit-vai-bo-PROD0125.html"
            },
            {
                "product_id": "PROD0126",
                "product_name": "Thịt cốt lết",
                "product_description": "Thịt cốt lết được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt cốt lết giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng.",
                "price": 149000,
                "discount": "56%",
                "url": "https://example.com/san-pham/thit/thit-cot-let-PROD0126.html"
            },
            {
                "product_id": "PROD0127",
                "product_name": "Sườn non heo",
                "product_description": "Sườn non heo được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Sườn non heo giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein.",
                "price": 196000,
                "discount": "65%",
                "url": "https://example.com/san-pham/thit/suon-non-heo-PROD0127.html"
            },
            {
                "product_id": "PROD0128",
                "product_name": "Lưỡi heo",
                "product_description": "Lưỡi heo được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Lưỡi heo giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm.",
                "price": 120000,
                "discount": "40%",
                "url": "https://example.com/san-pham/thit/luoi-heo-PROD0128.html"
            },
            {
                "product_id": "PROD0129",
                "product_name": "Thịt đùi gà góc tư",
                "product_description": "Thịt đùi gà góc tư được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt đùi gà góc tư giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein.",
                "price": 93000,
                "discount": "40%",
                "url": "https://example.com/san-pham/thit/thit-dui-ga-goc-tu-PROD0129.html"
            },
            {
                "product_id": "PROD0130",
                "product_name": "Thịt thăn lợn",
                "product_description": "Thịt thăn lợn được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt thăn lợn giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein.",
                "price": 154000,
                "discount": "65%",
                "url": "https://example.com/san-pham/thit/thit-than-lon-PROD0130.html"
            },
            {
                "product_id": "PROD0131",
                "product_name": "Lòng non heo",
                "product_description": "Lòng non heo được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Lòng non heo giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến.",
                "price": 113000,
                "discount": "56%",
                "url": "https://example.com/san-pham/thit/long-non-heo-PROD0131.html"
            },
            {
                "product_id": "PROD0132",
                "product_name": "Thịt bê non",
                "product_description": "Thịt bê non được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt bê non giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn.",
                "price": 289000,
                "discount": "40%",
                "url": "https://example.com/san-pham/thit/thit-be-non-PROD0132.html"
            },
            {
                "product_id": "PROD0133",
                "product_name": "Tim heo tươi",
                "product_description": "Tim heo tươi được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Tim heo tươi giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm.",
                "price": 105000,
                "discount": "11%",
                "url": "https://example.com/san-pham/thit/tim-heo-tuoi-PROD0133.html"
            },
            {
                "product_id": "PROD0134",
                "product_name": "Ba chỉ bò Mỹ",
                "product_description": "Ba chỉ bò Mỹ được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Ba chỉ bò Mỹ giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến.",
                "price": 320000,
                "discount": "26%",
                "url": "https://example.com/san-pham/thit/ba-chi-bo-my-PROD0134.html"
            },
            {
                "product_id": "PROD0135",
                "product_name": "Gà ta nguyên con",
                "product_description": "Gà ta nguyên con được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Gà ta nguyên con giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein.",
                "price": 178000,
                "discount": "20%",
                "url": "https://example.com/san-pham/thit/ga-ta-nguyen-con-PROD0135.html"
            },
            {
                "product_id": "PROD0136",
                "product_name": "Ức gà tươi CP",
                "product_description": "Ức gà tươi CP được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Ức gà tươi CP giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein.",
                "price": 85000,
                "discount": "40%",
                "url": "https://example.com/san-pham/thit/uc-ga-tuoi-cp-PROD0136.html"
            },
            {
                "product_id": "PROD0137",
                "product_name": "Thịt heo ba chỉ sạch",
                "product_description": "Thịt heo ba chỉ sạch được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt heo ba chỉ sạch giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm.",
                "price": 139000,
                "discount": "65%",
                "url": "https://example.com/san-pham/thit/thit-heo-ba-chi-sach-PROD0137.html"
            },
            {
                "product_id": "PROD0138",
                "product_name": "Thịt dê tơ",
                "product_description": "Thịt dê tơ được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt dê tơ giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến.",
                "price": 305000,
                "discount": "56%",
                "url": "https://example.com/san-pham/thit/thit-de-to-PROD0138.html"
            },
            {
                "product_id": "PROD0139",
                "product_name": "Thịt bò thăn Úc nhập khẩu",
                "product_description": "Thịt bò thăn Úc nhập khẩu được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt bò thăn Úc nhập khẩu giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm.",
                "price": 420000,
                "discount": "71%",
                "url": "https://example.com/san-pham/thit/thit-bo-than-uc-nhap-khau-PROD0139.html"
            },
            {
                "product_id": "PROD0140",
                "product_name": "Thịt vịt xiêm",
                "product_description": "Thịt vịt xiêm được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt vịt xiêm giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Được đóng gói hút chân không, tiện lợi cho việc bảo quản và sử dụng.",
                "price": 145000,
                "discount": "49%",
                "url": "https://example.com/san-pham/thit/thit-vit-xiem-PROD0140.html"
            },
            {
                "product_id": "PROD0141",
                "product_name": "Chân giò heo rút xương",
                "product_description": "Chân giò heo rút xương được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Chân giò heo rút xương giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Để có món ăn ngon nhất, nên rã đông tự nhiên trước khi chế biến. Thích hợp cho các món nướng, lẩu, xào, đảm bảo hương vị đậm đà. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein.",
                "price": 169000,
                "discount": "22%",
                "url": "https://example.com/san-pham/thit/chan-gio-heo-rut-xuong-PROD0141.html"
            },
            {
                "product_id": "PROD0142",
                "product_name": "Thịt băm hỗn hợp",
                "product_description": "Thịt băm hỗn hợp được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Thịt băm hỗn hợp giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Đây là nguyên liệu không thể thiếu cho những bữa ăn chất lượng, bổ dưỡng cho gia đình bạn. Nguồn gốc rõ ràng, đã qua kiểm dịch vệ sinh an toàn thực phẩm. Cung cấp đầy đủ dưỡng chất cần thiết cho cơ thể, đặc biệt là protein.",
                "price": 115000,
                "discount": "11%",
                "url": "https://example.com/san-pham/thit/thit-bam-hon-hop-PROD0142.html"
            }
        ]
    },
    {
        "category": "THUỶ HẢI SẢN",
        "products": [
            {
                "product_id": "PROD0143",
                "product_name": "Ốc hương",
                "product_description": "Ốc hương được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Ốc hương giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng.",
                "price": 280000,
                "discount": "51%",
                "url": "https://example.com/san-pham/thuy-hai-san/oc-huong-PROD0143.html"
            },
            {
                "product_id": "PROD0144",
                "product_name": "Sò điệp tươi",
                "product_description": "Sò điệp tươi được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Sò điệp tươi giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Thích hợp để hấp, nướng, lẩu, hoặc chế biến thành các món chiên. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng.",
                "price": 195000,
                "discount": "65%",
                "url": "https://example.com/san-pham/thuy-hai-san/so-diep-tuoi-PROD0144.html"
            },
            {
                "product_id": "PROD0145",
                "product_name": "Tôm càng xanh",
                "product_description": "Tôm càng xanh được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Tôm càng xanh giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng.",
                "price": 380000,
                "discount": "71%",
                "url": "https://example.com/san-pham/thuy-hai-san/tom-cang-xanh-PROD0145.html"
            },
            {
                "product_id": "PROD0146",
                "product_name": "Tôm sú tươi sống",
                "product_description": "Tôm sú tươi sống được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Tôm sú tươi sống giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình.",
                "price": 350000,
                "discount": "27%",
                "url": "https://example.com/san-pham/thuy-hai-san/tom-su-tuoi-song-PROD0146.html"
            },
            {
                "product_id": "PROD0147",
                "product_name": "Cá ngừ đại dương",
                "product_description": "Cá ngừ đại dương được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Cá ngừ đại dương giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng.",
                "price": 270000,
                "discount": "7%",
                "url": "https://example.com/san-pham/thuy-hai-san/ca-ngu-dai-duong-PROD0147.html"
            },
            {
                "product_id": "PROD0148",
                "product_name": "Hàu sữa Pháp",
                "product_description": "Hàu sữa Pháp được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Hàu sữa Pháp giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Thích hợp để hấp, nướng, lẩu, hoặc chế biến thành các món chiên.",
                "price": 180000,
                "discount": "4%",
                "url": "https://example.com/san-pham/thuy-hai-san/hau-sua-phap-PROD0148.html"
            },
            {
                "product_id": "PROD0149",
                "product_name": "Cá lóc đồng",
                "product_description": "Cá lóc đồng được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Cá lóc đồng giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Đảm bảo tươi sống hoặc cấp đông nhanh tại nguồn, giữ trọn vị biển. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình.",
                "price": 130000,
                "discount": "26%",
                "url": "https://example.com/san-pham/thuy-hai-san/ca-loc-dong-PROD0149.html"
            },
            {
                "product_id": "PROD0150",
                "product_name": "Cá hồi Nauy fillet",
                "product_description": "Cá hồi Nauy fillet được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Cá hồi Nauy fillet giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Đảm bảo tươi sống hoặc cấp đông nhanh tại nguồn, giữ trọn vị biển. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng.",
                "price": 380000,
                "discount": "40%",
                "url": "https://example.com/san-pham/thuy-hai-san/ca-hoi-nauy-fillet-PROD0150.html"
            },
            {
                "product_id": "PROD0151",
                "product_name": "Cua thịt Cà Mau",
                "product_description": "Cua thịt Cà Mau được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Cua thịt Cà Mau giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Thích hợp để hấp, nướng, lẩu, hoặc chế biến thành các món chiên. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng. Đảm bảo tươi sống hoặc cấp đông nhanh tại nguồn, giữ trọn vị biển.",
                "price": 550000,
                "discount": "20%",
                "url": "https://example.com/san-pham/thuy-hai-san/cua-thit-ca-mau-PROD0151.html"
            },
            {
                "product_id": "PROD0152",
                "product_name": "Bề bề rang muối",
                "product_description": "Bề bề rang muối được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Bề bề rang muối giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não.",
                "price": 250000,
                "discount": "62%",
                "url": "https://example.com/san-pham/thuy-hai-san/be-be-rang-muoi-PROD0152.html"
            },
            {
                "product_id": "PROD0153",
                "product_name": "Cá diêu hồng",
                "product_description": "Cá diêu hồng được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Cá diêu hồng giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng.",
                "price": 105000,
                "discount": "71%",
                "url": "https://example.com/san-pham/thuy-hai-san/ca-dieu-hong-PROD0153.html"
            },
            {
                "product_id": "PROD0154",
                "product_name": "Mực ống tươi",
                "product_description": "Mực ống tươi được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Mực ống tươi giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Thích hợp để hấp, nướng, lẩu, hoặc chế biến thành các món chiên. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng. Đảm bảo tươi sống hoặc cấp đông nhanh tại nguồn, giữ trọn vị biển.",
                "price": 220000,
                "discount": "40%",
                "url": "https://example.com/san-pham/thuy-hai-san/muc-ong-tuoi-PROD0154.html"
            },
            {
                "product_id": "PROD0155",
                "product_name": "Tôm thẻ chân trắng",
                "product_description": "Tôm thẻ chân trắng được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Tôm thẻ chân trắng giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình.",
                "price": 280000,
                "discount": "22%",
                "url": "https://example.com/san-pham/thuy-hai-san/tom-the-chan-trang-PROD0155.html"
            },
            {
                "product_id": "PROD0156",
                "product_name": "Lươn đồng",
                "product_description": "Lươn đồng được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Lươn đồng giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Thích hợp để hấp, nướng, lẩu, hoặc chế biến thành các món chiên. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng.",
                "price": 185000,
                "discount": "11%",
                "url": "https://example.com/san-pham/thuy-hai-san/luon-dong-PROD0156.html"
            },
            {
                "product_id": "PROD0157",
                "product_name": "Cá trắm đen",
                "product_description": "Cá trắm đen được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Cá trắm đen giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình.",
                "price": 160000,
                "discount": "40%",
                "url": "https://example.com/san-pham/thuy-hai-san/ca-tram-den-PROD0157.html"
            },
            {
                "product_id": "PROD0158",
                "product_name": "Chả mực Hạ Long",
                "product_description": "Chả mực Hạ Long được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Chả mực Hạ Long giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não.",
                "price": 170000,
                "discount": "1%",
                "url": "https://example.com/san-pham/thuy-hai-san/cha-muc-ha-long-PROD0158.html"
            },
            {
                "product_id": "PROD0159",
                "product_name": "Bạch tuộc tươi",
                "product_description": "Bạch tuộc tươi được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Bạch tuộc tươi giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình.",
                "price": 210000,
                "discount": "71%",
                "url": "https://example.com/san-pham/thuy-hai-san/bach-tuoc-tuoi-PROD0159.html"
            },
            {
                "product_id": "PROD0160",
                "product_name": "Ngao trắng",
                "product_description": "Ngao trắng được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Ngao trắng giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Thích hợp để hấp, nướng, lẩu, hoặc chế biến thành các món chiên. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng.",
                "price": 95000,
                "discount": "55%",
                "url": "https://example.com/san-pham/thuy-hai-san/ngao-trang-PROD0160.html"
            },
            {
                "product_id": "PROD0161",
                "product_name": "Ghẹ xanh sống",
                "product_description": "Ghẹ xanh sống được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Ghẹ xanh sống giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Đảm bảo tươi sống hoặc cấp đông nhanh tại nguồn, giữ trọn vị biển.",
                "price": 450000,
                "discount": "65%",
                "url": "https://example.com/san-pham/thuy-hai-san/ghe-xanh-song-PROD0161.html"
            },
            {
                "product_id": "PROD0162",
                "product_name": "Mực một nắng",
                "product_description": "Mực một nắng được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Mực một nắng giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng.",
                "price": 280000,
                "discount": "65%",
                "url": "https://example.com/san-pham/thuy-hai-san/muc-mot-nang-PROD0162.html"
            },
            {
                "product_id": "PROD0163",
                "product_name": "Cá chẽm",
                "product_description": "Cá chẽm được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Cá chẽm giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Đảm bảo tươi sống hoặc cấp đông nhanh tại nguồn, giữ trọn vị biển. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng.",
                "price": 190000,
                "discount": "40%",
                "url": "https://example.com/san-pham/thuy-hai-san/ca-chem-PROD0163.html"
            },
            {
                "product_id": "PROD0164",
                "product_name": "Ếch đồng làm sạch",
                "product_description": "Ếch đồng làm sạch được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Ếch đồng làm sạch giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Thích hợp để hấp, nướng, lẩu, hoặc chế biến thành các món chiên.",
                "price": 170000,
                "discount": "11%",
                "url": "https://example.com/san-pham/thuy-hai-san/ech-dong-lam-sach-PROD0164.html"
            },
            {
                "product_id": "PROD0165",
                "product_name": "Hải sâm",
                "product_description": "Hải sâm được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Hải sâm giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình.",
                "price": 600000,
                "discount": "65%",
                "url": "https://example.com/san-pham/thuy-hai-san/hai-sam-PROD0165.html"
            },
            {
                "product_id": "PROD0166",
                "product_name": "Bào ngư tươi",
                "product_description": "Bào ngư tươi được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Bào ngư tươi giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng. Thích hợp để hấp, nướng, lẩu, hoặc chế biến thành các món chiên.",
                "price": 750000,
                "discount": "56%",
                "url": "https://example.com/san-pham/thuy-hai-san/bao-ngu-tuoi-PROD0166.html"
            },
            {
                "product_id": "PROD0167",
                "product_name": "Cá thu cắt khúc",
                "product_description": "Cá thu cắt khúc được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Cá thu cắt khúc giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Đảm bảo tươi sống hoặc cấp đông nhanh tại nguồn, giữ trọn vị biển.",
                "price": 175000,
                "discount": "64%",
                "url": "https://example.com/san-pham/thuy-hai-san/ca-thu-cat-khuc-PROD0167.html"
            },
            {
                "product_id": "PROD0168",
                "product_name": "Cá basa fillet đông lạnh",
                "product_description": "Cá basa fillet đông lạnh được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Cá basa fillet đông lạnh giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình.",
                "price": 105000,
                "discount": "75%",
                "url": "https://example.com/san-pham/thuy-hai-san/ca-basa-fillet-dong-lanh-PROD0168.html"
            },
            {
                "product_id": "PROD0169",
                "product_name": "Ốc hương",
                "product_description": "Ốc hương được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Ốc hương giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não.",
                "price": 280000,
                "discount": "14%",
                "url": "https://example.com/san-pham/thuy-hai-san/oc-huong-PROD0169.html"
            },
            {
                "product_id": "PROD0170",
                "product_name": "Sò điệp tươi",
                "product_description": "Sò điệp tươi được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Sò điệp tươi giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Thích hợp để hấp, nướng, lẩu, hoặc chế biến thành các món chiên. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng.",
                "price": 195000,
                "discount": "17%",
                "url": "https://example.com/san-pham/thuy-hai-san/so-diep-tuoi-PROD0170.html"
            },
            {
                "product_id": "PROD0171",
                "product_name": "Tôm càng xanh",
                "product_description": "Tôm càng xanh được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Tôm càng xanh giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Đảm bảo tươi sống hoặc cấp đông nhanh tại nguồn, giữ trọn vị biển. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não.",
                "price": 370000,
                "discount": "1%",
                "url": "https://example.com/san-pham/thuy-hai-san/tom-cang-xanh-PROD0171.html"
            },
            {
                "product_id": "PROD0172",
                "product_name": "Tôm sú tươi sống",
                "product_description": "Tôm sú tươi sống được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Tôm sú tươi sống giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Thích hợp để hấp, nướng, lẩu, hoặc chế biến thành các món chiên.",
                "price": 350000,
                "discount": "26%",
                "url": "https://example.com/san-pham/thuy-hai-san/tom-su-tuoi-song-PROD0172.html"
            },
            {
                "product_id": "PROD0173",
                "product_name": "Cá ngừ đại dương",
                "product_description": "Cá ngừ đại dương được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Cá ngừ đại dương giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Đảm bảo tươi sống hoặc cấp đông nhanh tại nguồn, giữ trọn vị biển. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng.",
                "price": 270000,
                "discount": "20%",
                "url": "https://example.com/san-pham/thuy-hai-san/ca-ngu-dai-duong-PROD0173.html"
            },
            {
                "product_id": "PROD0174",
                "product_name": "Hàu sữa Pháp",
                "product_description": "Hàu sữa Pháp được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Hàu sữa Pháp giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Thích hợp để hấp, nướng, lẩu, hoặc chế biến thành các món chiên. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng.",
                "price": 180000,
                "discount": "40%",
                "url": "https://example.com/san-pham/thuy-hai-san/hau-sua-phap-PROD0174.html"
            },
            {
                "product_id": "PROD0175",
                "product_name": "Cá lóc đồng",
                "product_description": "Cá lóc đồng được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Cá lóc đồng giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Đảm bảo tươi sống hoặc cấp đông nhanh tại nguồn, giữ trọn vị biển.",
                "price": 130000,
                "discount": "71%",
                "url": "https://example.com/san-pham/thuy-hai-san/ca-loc-dong-PROD0175.html"
            },
            {
                "product_id": "PROD0176",
                "product_name": "Cá hồi Nauy fillet",
                "product_description": "Cá hồi Nauy fillet được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Cá hồi Nauy fillet giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng. Thích hợp để hấp, nướng, lẩu, hoặc chế biến thành các món chiên.",
                "price": 380000,
                "discount": "65%",
                "url": "https://example.com/san-pham/thuy-hai-san/ca-hoi-nauy-fillet-PROD0176.html"
            },
            {
                "product_id": "PROD0177",
                "product_name": "Cua thịt Cà Mau",
                "product_description": "Cua thịt Cà Mau được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Cua thịt Cà Mau giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình.",
                "price": 550000,
                "discount": "4%",
                "url": "https://example.com/san-pham/thuy-hai-san/cua-thit-ca-mau-PROD0177.html"
            },
            {
                "product_id": "PROD0178",
                "product_name": "Bề bề rang muối",
                "product_description": "Bề bề rang muối được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Bề bề rang muối giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng. Thích hợp để hấp, nướng, lẩu, hoặc chế biến thành các món chiên.",
                "price": 250000,
                "discount": "6%",
                "url": "https://example.com/san-pham/thuy-hai-san/be-be-rang-muoi-PROD0178.html"
            },
            {
                "product_id": "PROD0179",
                "product_name": "Cá diêu hồng",
                "product_description": "Cá diêu hồng được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Cá diêu hồng giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não.",
                "price": 105000,
                "discount": "51%",
                "url": "https://example.com/san-pham/thuy-hai-san/ca-dieu-hong-PROD0179.html"
            },
            {
                "product_id": "PROD0180",
                "product_name": "Mực ống tươi",
                "product_description": "Mực ống tươi được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Mực ống tươi giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng.",
                "price": 220000,
                "discount": "40%",
                "url": "https://example.com/san-pham/thuy-hai-san/muc-ong-tuoi-PROD0180.html"
            },
            {
                "product_id": "PROD0181",
                "product_name": "Tôm thẻ chân trắng",
                "product_description": "Tôm thẻ chân trắng được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Tôm thẻ chân trắng giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng. Thích hợp để hấp, nướng, lẩu, hoặc chế biến thành các món chiên.",
                "price": 270000,
                "discount": "22%",
                "url": "https://example.com/san-pham/thuy-hai-san/tom-the-chan-trang-PROD0181.html"
            },
            {
                "product_id": "PROD0182",
                "product_name": "Lươn đồng",
                "product_description": "Lươn đồng được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Lươn đồng giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Đảm bảo tươi sống hoặc cấp đông nhanh tại nguồn, giữ trọn vị biển.",
                "price": 185000,
                "discount": "11%",
                "url": "https://example.com/san-pham/thuy-hai-san/luon-dong-PROD0182.html"
            },
            {
                "product_id": "PROD0183",
                "product_name": "Cá trắm đen",
                "product_description": "Cá trắm đen được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Cá trắm đen giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Thích hợp để hấp, nướng, lẩu, hoặc chế biến thành các món chiên.",
                "price": 160000,
                "discount": "40%",
                "url": "https://example.com/san-pham/thuy-hai-san/ca-tram-den-PROD0183.html"
            },
            {
                "product_id": "PROD0184",
                "product_name": "Chả mực Hạ Long",
                "product_description": "Chả mực Hạ Long được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Chả mực Hạ Long giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối.",
                "price": 170000,
                "discount": "71%",
                "url": "https://example.com/san-pham/thuy-hai-san/cha-muc-ha-long-PROD0184.html"
            },
            {
                "product_id": "PROD0185",
                "product_name": "Bạch tuộc tươi",
                "product_description": "Bạch tuộc tươi được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Bạch tuộc tươi giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Thích hợp để hấp, nướng, lẩu, hoặc chế biến thành các món chiên. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng.",
                "price": 210000,
                "discount": "55%",
                "url": "https://example.com/san-pham/thuy-hai-san/bach-tuoc-tuoi-PROD0185.html"
            },
            {
                "product_id": "PROD0186",
                "product_name": "Ngao trắng",
                "product_description": "Ngao trắng được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Ngao trắng giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Đảm bảo tươi sống hoặc cấp đông nhanh tại nguồn, giữ trọn vị biển.",
                "price": 95000,
                "discount": "65%",
                "url": "https://example.com/san-pham/thuy-hai-san/ngao-trang-PROD0186.html"
            },
            {
                "product_id": "PROD0187",
                "product_name": "Ghẹ xanh sống",
                "product_description": "Ghẹ xanh sống được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Ghẹ xanh sống giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng. Thích hợp để hấp, nướng, lẩu, hoặc chế biến thành các món chiên.",
                "price": 450000,
                "discount": "56%",
                "url": "https://example.com/san-pham/thuy-hai-san/ghe-xanh-song-PROD0187.html"
            },
            {
                "product_id": "PROD0188",
                "product_name": "Mực một nắng",
                "product_description": "Mực một nắng được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Mực một nắng giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối.",
                "price": 280000,
                "discount": "64%",
                "url": "https://example.com/san-pham/thuy-hai-san/muc-mot-nang-PROD0188.html"
            },
            {
                "product_id": "PROD0189",
                "product_name": "Cá chẽm",
                "product_description": "Cá chẽm được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Cá chẽm giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng. Thích hợp để hấp, nướng, lẩu, hoặc chế biến thành các món chiên.",
                "price": 190000,
                "discount": "75%",
                "url": "https://example.com/san-pham/thuy-hai-san/ca-chem-PROD0189.html"
            },
            {
                "product_id": "PROD0190",
                "product_name": "Ếch đồng làm sạch",
                "product_description": "Ếch đồng làm sạch được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Ếch đồng làm sạch giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Đảm bảo tươi sống hoặc cấp đông nhanh tại nguồn, giữ trọn vị biển.",
                "price": 170000,
                "discount": "14%",
                "url": "https://example.com/san-pham/thuy-hai-san/ech-dong-lam-sach-PROD0190.html"
            },
            {
                "product_id": "PROD0191",
                "product_name": "Hải sâm",
                "product_description": "Hải sâm được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Hải sâm giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng. Thích hợp để hấp, nướng, lẩu, hoặc chế biến thành các món chiên.",
                "price": 600000,
                "discount": "17%",
                "url": "https://example.com/san-pham/thuy-hai-san/hai-sam-PROD0191.html"
            },
            {
                "product_id": "PROD0192",
                "product_name": "Bào ngư tươi",
                "product_description": "Bào ngư tươi được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Bào ngư tươi giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Đảm bảo tươi sống hoặc cấp đông nhanh tại nguồn, giữ trọn vị biển.",
                "price": 750000,
                "discount": "1%",
                "url": "https://example.com/san-pham/thuy-hai-san/bao-ngu-tuoi-PROD0192.html"
            },
            {
                "product_id": "PROD0193",
                "product_name": "Cá thu cắt khúc",
                "product_description": "Cá thu cắt khúc được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Cá thu cắt khúc giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Nguồn gốc từ các vùng biển sạch, an toàn tuyệt đối. Thích hợp để hấp, nướng, lẩu, hoặc chế biến thành các món chiên. Sản phẩm được giao hàng nhanh chóng, đảm bảo độ tươi ngon khi đến tay khách hàng.",
                "price": 175000,
                "discount": "26%",
                "url": "https://example.com/san-pham/thuy-hai-san/ca-thu-cat-khuc-PROD0193.html"
            },
            {
                "product_id": "PROD0194",
                "product_name": "Cá basa fillet đông lạnh",
                "product_description": "Cá basa fillet đông lạnh được tuyển chọn kỹ lưỡng từ nguồn cung cấp đạt chuẩn, đảm bảo vệ sinh an toàn thực phẩm. Với quy trình sơ chế và bảo quản hiện đại, sản phẩm giữ được độ tươi ngon tự nhiên, không bị biến chất. Cá basa fillet đông lạnh giàu protein và các dưỡng chất thiết yếu, rất phù hợp để chế biến thành nhiều món ăn hấp dẫn như nướng, chiên, xào, nấu canh. Hải sản tươi ngon mỗi ngày, mang đến bữa ăn thịnh soạn cho gia đình. Cung cấp omega-3 dồi dào, tốt cho tim mạch và trí não. Đảm bảo tươi sống hoặc cấp đông nhanh tại nguồn, giữ trọn vị biển.",
                "price": 105000,
                "discount": "20%",
                "url": "https://example.com/san-pham/thuy-hai-san/ca-basa-fillet-dong-lanh-PROD0194.html"
            }
